In [207]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression


%matplotlib inline

In [208]:
df = pd.read_csv('../Data/Players With Award Data/complete_players_with_award_voting.csv',index_col=0)

df = df.drop(columns=['STAR','DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
       'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
       'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
       'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
       'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
       'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
       'SMOTY'])

In [209]:
yr = int(input('Year = '))
# train_set, test_set = train_test_split(df, test_size=0.2, random_state=0)
test_set = df[(df['Year'] == yr)]
train_set = df.drop(test_set.index)

x_train = train_set.drop(columns=['MVP','MVP_Share','MVP_Rank'])
y_train = train_set[['MVP_Share']]
x_test = test_set.drop(columns=['MVP','MVP_Share','MVP_Rank'])
y_test = test_set['MVP_Share']
y_test_w_name = test_set[['Player','Year','Tm','MVP_Share','MVP','MVP_Rank']]


train_set.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Tm_AGE',
       'Tm_W', 'Tm_L', 'Tm_PW', 'Tm_PL', 'Tm_MOV', 'Tm_SOS', 'Tm_SRS',
       'Tm_ORtg', 'Tm_DRtg', 'Tm_NRtg', 'Tm_PACE', 'Tm_FTr', 'Tm_3PAr',
       'Tm_TS%', 'Tm_eFG%', 'Tm_TOV%', 'Tm_ORB%', 'Tm_FT/FGA', 'Tm_Op_eFG%',
       'Tm_Op_TOV%', 'Tm_DRB%', 'Tm_Op_FT/FGA', 'Win%', 'MVP_Rank',
       'MVP_Share', 'MVP'],
      dtype='object')

In [210]:
cm = train_set.corr().MVP_Share.apply(np.absolute).sort_values(ascending=False)

In [211]:
list(cm[cm > .1].to_dict().keys())

['MVP_Share',
 'MVP',
 'FTA',
 'FT',
 'PTS',
 'FG',
 '2P',
 'FGA',
 '2PA',
 'TOV',
 'DRB',
 'AST',
 'TRB',
 'STL',
 'GS',
 'MP',
 'BLK',
 'MVP_Rank']

In [212]:
feature_list = ['MP', 'PTS', 'FG', 'FGA', 'TOV', 'AST', 'STL', 'TRB', 'BLK']

x_train = x_train[feature_list]
x_test = x_test[feature_list]

In [213]:
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
        ('linear_reg',LinearRegression())
    ])
ppln_fitted = num_pipeline.fit(x_train,y_train)

In [214]:
y_pred = ppln_fitted.predict(x_test)

y_test_w_name['pred'] = y_pred
y_test_w_name['pred_rank'] = y_test_w_name.pred.rank(ascending=False).astype("int")

/var/folders/0p/ttyhqytj0zxf3tq1njp37rqw0000gn/T/ipykernel_1024/1015442088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test_w_name['pred'] = y_pred
/var/folders/0p/ttyhqytj0zxf3tq1njp37rqw0000gn/T/ipykernel_1024/1015442088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test_w_name['pred_rank'] = y_test_w_name.pred.rank(ascending=False).astype("int")


In [215]:
y_test_w_name.sort_values('pred_rank',ascending=True).head(10)

,Player,Year,Tm,MVP_Share,MVP,MVP_Rank,pred,pred_rank
12117,LeBron James,2010,CLE,0.9797,True,1.0,0.163018,1
11839,Kevin Durant,2010,OKC,0.4951,False,2.0,0.112980,2
11809,Dwyane Wade,2010,MIA,0.0967,False,5.0,0.112702,3
11995,Chris Bosh,2010,TOR,0.0008,False,12.0,0.087546,4
12082,Chris Paul,2010,NOH,0.0000,False,0.0,0.085788,5
12142,Dwight Howard,2010,ORL,0.3886,False,4.0,0.085140,6
12023,Dirk Nowitzki,2010,DAL,0.0447,False,7.0,0.083761,7
11954,Carmelo Anthony,2010,DEN,0.0528,False,6.0,0.076400,8
11804,Deron Williams,2010,UTA,0.0057,False,9.0,0.074852,9
11963,Chauncey Billups,2010,DEN,0.0008,False,12.0,0.070365,10


In [206]:
df.Year.max()

2018